# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸（以下、「簡易外皮入力モデル」）における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、入力値である部位種別の熱貫流率をもとに、部位のU値から壁体構成を決定する方法により決定する。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は、外皮性能計算プログラムの入力情報に用途別床面積を追加したものと同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 簡易外皮入力モデル住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 簡易外皮入力モデル住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 簡易外皮入力モデル住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

## 2. コンバート

### 1) 共通

- 地域の区分、主たる居室の床面積(m<sup>2</sup>)、その他の床面積(m<sup>2</sup>)、床面積の合計(m<sup>2</sup>)の情報は維持する。
- 新たに変数「簡易外皮入力フラグ」を作成し、LV1から生成したLV2情報は必ずTrueとする。

#### Input

```
data = {
    'Region'                     : # 地域の区分 1 ~ 8 の整数
    'MainOccupantRoomFloorArea'  : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea' : # その他の居室の床面積(m2)
    'TotalFloorArea'             : # 床面積の合計(m2)
}
```

#### Output

```
data = {
    'Region'                      : # 地域の区分 1 ~ 8 の整数
    'IsSimplifiedInput'           : # bool値 簡易入力から生成したデータか否かを表す, LV1からの場合は常にTrueとなる。
    'MainOccupantRoomFloorArea'   : # 主たる居室の床面積(m2)
    'OtherOccupantRoomFloorArea'  : # その他の居室の床面積(m2)
    'TotalFloorArea'              : # 床面積の合計(m2)
}
```

#### Function

In [1]:
def convert_common(d):
    return {
        'Region'                      : d['Region'],
        'FlagSimplifiedEnvelopeInput' : True,
        'MainOccupantRoomFloorArea'   : d['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloorArea'  : d['OtherOccupantRoomFloorArea'],
        'TotalFloorArea'              : d['TotalFloorArea']
    }

#### Example

In [2]:
convert_common({
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0
})

{'FlagSimplifiedEnvelopeInput': True,
 'MainOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloorArea': 60.0,
 'Region': 6,
 'TotalFloorArea': 120.0}

### 2) 開口部以外の不透明部位

- 部位のU値から壁体構成を決定する

#### Input

```
data = {
    'InsulationType'                  : # 断熱構造 string値,  'floor_ins' or 'base_ins'
    'U_roof'                          : # 屋根・天井の熱貫流率(W/m2K)
    'U_wall'                          : # 外壁の熱貫流率(W/m2K)
    'U_floor'                         : # 床の熱貫流率(W/m2K)
    'U_door'                          : # ドアの熱貫流率(W/m2K)
    'DoesInputEarthFloorPerimeterPsi' : # 土間床等の外周部の線熱貫流率を入力するかどうか, bool値
    'Psi_base_others'                 : # その他の土間床等の外周部の線熱貫流率(W/mK),
                                          'DoesInputEarthFloorPerimeterPsi'がFalseの場合は無視される。
    'Psi_base_entrance'               : # 玄関等の土間床等の外周部の線熱貫流率(W/mK)
                                          'DoesInputEarthFloorPerimeterPsi'がFalseの場合は無視される。
}
```

### Output

```
data = {
    'name'                        : # 壁体名称
    'structure'                   : # 一般部位構造種別 1 ~ 4 の整数
    'type'                        : # 壁体種別 1 ~ 6 の整数
    'Layers'                      : {
        'matrial'                 : # 部材名称
        'd'                       : # 厚さ(m)
        'lambda'                  : # 熱伝導率(W/(m・K))
        'Crho'                    : # 容積比熱(J/(m3・K))
    }
    'Ua'                          : # 平均熱貫流率(W/(m2・K))
}
```

### Function

In [3]:
def convert_wall(d1):
    import nbimporter    
    from U_Value_to_Detail_Specification import convert_UtoDetailSpecification
    Layers=convert_UtoDetailSpecification(d1)
   
    return {'structure': d1['structure'],
            'name': d1['name'],
            'type': d1['type'],
            'Ua': d1['Ua'],
            'Layers':Layers}

### Example

In [4]:
convert_wall( {
    'structure':1,
    'name': 'Sample Wall',
    'type': 2,
    'Ua': 0.538})

Importing Jupyter notebook from U_Value_to_Detail_Specification.ipynb


{'Layers': {'Layers': [{'Cρ': 13, 'd': 0.074, 'material': 'GW16K', 'λ': 0.045},
   {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
 'Ua': 0.538,
 'name': 'Sample Wall',
 'structure': 1,
 'type': 2}

## 2-3. コンバート【分類：開口部】

- 開口部名称、日射熱取得率[-]、熱貫流率[W/(m<sup>2</sup>・K)]の情報は維持する。

### Input

```
data = {
    'name'                        : # 開口部名称
    'etaCooling'                  : # 日射熱取得率（夏期）(-)
    'etaHeating'                  : # 日射熱取得率（冬期）(-)
    'U'                           : # 熱貫流率(W/(m2・K))
}
```

### Output

```
data = {
    'name'                        : # 開口部名称
    'etaCooling'                  : # 日射熱取得率（夏期）(-)
    'etaHeating'                  : # 日射熱取得率（冬期）(-)
    'U'                           : # 熱貫流率(W/(m2・K))
}
```

### Function

In [5]:
def convert_window(d1):
    return {'name': d1['name'],
            'etac': d1['etac'],
            'etah': d1['etah'],
            'U': d1['U']}

### Example

In [6]:
convert_window({
    'name': 'Sample Window',
    'etac': 0.738,
    'etah': 0.738,
    'U': 4.65})

{'U': 4.65, 'etac': 0.738, 'etah': 0.738, 'name': 'Sample Window'}

## 2-4. コンバート【分類：外部日除け】

- 外部日除け名称、取得日射熱補正係数[-]の情報は維持する。

### Input

```
data = {
    'name'                        : # 外部日除け名称
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)
}
```

### Output

```
data = {
    'name'                        : # 外部日除け名称
    'fCooling'                    : # 取得日射熱補正係数（夏期）(-)
    'fHeating'                    : # 取得日射熱補正係数（冬期）(-)
}
```

### Function

In [7]:
def convert_sunshade(d1):
    return {'name': d1['name'],
            'fc': d1['fc'],
            'fh': d1['fh']}

### Example

In [8]:
convert_sunshade({
    'name': 'Sample Sunshade',
    'fc': 0.700,
    'fh': 0.600,})

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

## 2-5. コンバート【分類：部位情報】

- 簡易外皮入力モデル住戸の部位の長さおよび面積、設計住戸の延床面積より、設計住戸の各部位の長さおよび面積の推定を行う。

### Input

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Output

```
data = {
    'name'                        : # 壁体・開口部名称
    'FlagPerimeter'               : # 外皮フラグ 1 ~ 2 の整数
    'FlagSolarRadiation'          : # 日射フラグ 1 ~ 2 の整数
    'direction'                   : # 方位 1 ~ 14 の整数
    'FlagUnsteady'                : # 非定常フラグ 1 ~ 2 の整数
    'A'                           : # 部位面積(m2)
    'L'                           : # 部位長さ(m)
    'sunshade'                    : # 外部日除け名称
}
```

### Function

In [9]:
def convert_element(d1,d):
    FloorAreaS=90
    FloorArea=d['Common']['TotalFloorArea']
    return {'name': d1['name'],
            'FlagPerimeter': d1['FlagPerimeter'],
            'FlagSolarRadiation': d1['FlagSolarRadiation'],
            'FlagUnsteady': d1['FlagUnsteady'],
            'direction': d1['direction'],
            'AorL': round(d1['AorL']*FloorArea/FloorAreaS,2)}

### Example

In [10]:
convert_element({
    'name': 'Sample Element',
    'FlagPerimeter': 1,
    'FlagSolarRadiation': 1,
    'FlagUnsteady': 1,
    'direction': 1,
    'AorL': 1},{
    'Common':{
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloorArea': 60.0,
    'TotalFloorArea': 120.0,}})

{'AorL': 1.33,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'direction': 1,
 'name': 'Sample Element'}

## 2-6. コンバート【分類：簡易外皮入力】

- 断熱構造、屋根・天井熱貫流率[W/(m<sup>2</sup>・K)]、外壁熱貫流率[W/(m<sup>2</sup>・K)]、床熱貫流率[W/(m<sup>2</sup>・K)]、ドア熱貫流率[W/(m<sup>2</sup>・K)]、垂直面日射熱取得率[-]、取得日射熱補正係数の入力[-]、取得日射熱補正係数[-]、土間床等の外周部の入力、一般線熱貫流率[W/(m・K)]、玄関等線熱貫流率[W/(m・K)]の情報は維持する。

### Input

```
data = {
    'InsulationStructure'        : # 床/基礎の断熱構造 1 ~ 2 の整数
    'UCeiling'                   : # 屋根・天井熱貫流率(W/(m2・K))
    'UWall'                      : # 外壁熱貫流率(W/(m2・K))
    'UFloor'                     : # 床熱貫流率(W/(m2・K))
    'UDoor'                      : # ドア熱貫流率(W/(m2・K))
    'UWindow'                    : # 窓熱貫流率(W/(m2・K))
    'etaWindowCooling'           : # 垂直面日射熱取得率（夏期）   
    'etaWindowHeating'           : # 垂直面日射熱取得率（冬期）
    'FlagfWinowInput'            : # 取得日射熱補正係数の入力の入力 1 ~ 2 の整数
    'fWindowCooling'             : # 取得日射熱補正係数（夏期）
    'fWindowHeating'             : # 取得日射熱補正係数（冬期）
    'FlagPerimeterInput'         : # 土間床等の外周部の入力 1 ~ 2 の整数
    'psiPerimeter'               : # 一般線熱貫流率(W/(m・K))
    'psiPerimeterDoor'           : # 玄関等線熱貫流率(W/(m・K))
}
```

### Output

```
data = {
    'InsulationStructure'        : # 床/基礎の断熱構造 1 ~ 2 の整数
    'UCeiling'                   : # 屋根・天井熱貫流率(W/(m2・K))
    'UWall'                      : # 外壁熱貫流率(W/(m2・K))
    'UFloor'                     : # 床熱貫流率(W/(m2・K))
    'UDoor'                      : # ドア熱貫流率(W/(m2・K))
    'UWindow'                    : # 窓熱貫流率(W/(m2・K))
    'etaWindowCooling'           : # 垂直面日射熱取得率（夏期）   
    'etaWindowHeating'           : # 垂直面日射熱取得率（冬期）
    'FlagfWinowInput'            : # 取得日射熱補正係数の入力の入力 1 ~ 2 の整数
    'fWindowCooling'             : # 取得日射熱補正係数（夏期）
    'fWindowHeating'             : # 取得日射熱補正係数（冬期）
    'FlagPerimeterInput'         : # 土間床等の外周部の入力 1 ~ 2 の整数
    'psiPerimeter'               : # 一般線熱貫流率(W/(m・K))
    'psiPerimeterDoor'           : # 玄関等線熱貫流率(W/(m・K))
}
```

### Function

In [11]:
def convert_SimplifiedEnvelopeInput(d): 
    
    if d['FlagPerimeterInput']==2:
        d['psiPerimeter']=1.8
        d['psiPerimeterDoor']=1.8
    
    return {
    'InsulationStructure'        : d['InsulationStructure'],
    'UCeiling'                   : d['UCeiling'],
    'UWall'                      : d['UWall'],
    'UFloor'                     : d['UFloor'],
    'UDoor'                      : d['UDoor'],
    'UWindow'                    : d['UWindow'],
    'etaWindowCooling'           : d['etaWindowCooling'],  
    'etaWindowHeating'           : d['etaWindowHeating'],
    'FlagfWinowInput'            : d['FlagfWinowInput'],
    'fWindowCooling'             : d['fWindowCooling'],
    'fWindowHeating'             : d['fWindowHeating'],
    'FlagPerimeterInput'         : d['FlagPerimeterInput'],
    'psiPerimeter'               : d['psiPerimeter'],
    'psiPerimeterDoor'           : d['psiPerimeterDoor']
    }

### Example

In [12]:
convert_SimplifiedEnvelopeInput({
    'InsulationStructure'        : 1,
    'UCeiling'                   : 0.24,
    'UWall'                      : 0.53,
    'UFloor'                     : 0.48,
    'UDoor'                      : 2.33,
    'UWindow'                    : 3.49,
    'etaWindowCooling'           : 0.51,  
    'etaWindowHeating'           : 0.51,
    'FlagfWinowInput'            : 1,
    'fWindowCooling'             : 0.70,
    'fWindowHeating'             : 0.60,
    'FlagPerimeterInput'         : 2,
    'psiPerimeter'               : 1.80,
    'psiPerimeterDoor'           : 1.80
})

{'FlagPerimeterInput': 2,
 'FlagfWinowInput': 1,
 'InsulationStructure': 1,
 'UCeiling': 0.24,
 'UDoor': 2.33,
 'UFloor': 0.48,
 'UWall': 0.53,
 'UWindow': 3.49,
 'etaWindowCooling': 0.51,
 'etaWindowHeating': 0.51,
 'fWindowCooling': 0.7,
 'fWindowHeating': 0.6,
 'psiPerimeter': 1.8,
 'psiPerimeterDoor': 1.8}

## 2-7. コンバート【全分類】

In [13]:
def Convert(d):
    d_common = convert_common(d['Common'])
    d_SimplifiedEnvelopeInput=convert_SimplifiedEnvelopeInput(d['SimplifiedEnvelopeInput'])

    d_ceiling=convert_wall( {'structure':1,'name': 'Ceiling','type': 1,'Ua': d_SimplifiedEnvelopeInput['UCeiling']})
    d_wall=convert_wall( {'structure':1,'name': 'Wall','type': 2,'Ua': d_SimplifiedEnvelopeInput['UWall']})
    d_floor=convert_wall( {'structure':1,'name': 'Floor','type': 3,'Ua': d_SimplifiedEnvelopeInput['UFloor']})
    d_window=convert_window({'name': 'Window','etac': d_SimplifiedEnvelopeInput['etaWindowCooling'],'etah': d_SimplifiedEnvelopeInput['etaWindowHeating'],'U': d_SimplifiedEnvelopeInput['UWindow']})
    d_door=convert_window({'name': 'Door','etac':round(d_SimplifiedEnvelopeInput['UDoor']*0.034,3),'etah':round(d_SimplifiedEnvelopeInput['UDoor']*0.034,3),'U': d_SimplifiedEnvelopeInput['UDoor']})
    d_sunshade=convert_sunshade({'name': 'Sunshade','fc': d_SimplifiedEnvelopeInput['fWindowCooling'],'fh': d_SimplifiedEnvelopeInput['fWindowHeating']})

    #番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
    #ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade
    #標準住戸における部位の面積及び土間床等の外周部の長さ等（い）床断熱住戸の場合
    if d_SimplifiedEnvelopeInput['InsulationStructure']==1:
        StandardHouseElemntsFI = [ {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 1,'FlagUnsteady':1,'name':'Ceiling', 'AorL':50.85,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'Wall',    'AorL':30.47,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'Wall',    'AorL':22.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'Wall',    'AorL':47.92,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'Wall',    'AorL':22.28,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Door',    'AorL': 1.89,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Door',    'AorL': 1.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Window',  'AorL':22.69,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Window',  'AorL': 2.38,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Window',  'AorL': 3.63,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Window',  'AorL': 4.37,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'Floor',   'AorL':45.05,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbaseSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbaseNW', 'AorL': 0.91,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbaseNE', 'AorL': 0.91,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbaseSE', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbaseIF', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbasedSW','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbasedNW','AorL': 0.33,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbasedNE','AorL': 0.25,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbasedSE','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbasedIF','AorL': 0.57,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmSW',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmNW',  'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmNE',  'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'LprmSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmIF',  'AorL': 3.64,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmdSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmdNW', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmdNE', 'AorL': 1.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'prmdSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmdIF', 'AorL': 3.19,'Sunshade':None} ]
    #標準住戸における部位の面積及び土間床等の外周部の長さ等（ろ）基礎断熱住戸の場合
    elif d_SimplifiedEnvelopeInput['InsulationStructure']==2:
        StandardHouseElemntsFI = [ {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 1,'FlagUnsteady':1,'name':'Ceiling', 'AorL':50.85,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'Wall',    'AorL':30.47,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'Wall',    'AorL':22.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'Wall',    'AorL':47.92,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'Wall',    'AorL':22.28,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Door',    'AorL': 1.89,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Door',    'AorL': 1.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Door',    'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':2,'name':'Window',  'AorL':22.69,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':2,'name':'Window',  'AorL': 2.38,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':2,'name':'Window',  'AorL': 3.63,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':2,'name':'Window',  'AorL': 4.37,'Sunshade':'Sunshade'},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'Floor',   'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbaseSW', 'AorL': 5.30,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbaseNW', 'AorL': 1.48,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbaseNE', 'AorL': 4.62,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbaseSE', 'AorL': 2.40,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbaseIF', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'AbasedSW','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'AbasedNW','AorL': 0.33,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'AbasedNE','AorL': 0.25,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'AbasedSE','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'AbasedIF','AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmSW',  'AorL':10.61,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmNW',  'AorL': 2.97,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmNE',  'AorL': 9.24,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'LprmSE',  'AorL': 4.79,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmIF',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 7,'FlagUnsteady':1,'name':'LprmdSW', 'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 9,'FlagUnsteady':1,'name':'LprmdNW', 'AorL': 1.82,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 3,'FlagUnsteady':1,'name':'LprmdNE', 'AorL': 1.37,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':1,'direction': 5,'FlagUnsteady':1,'name':'prmdSE',  'AorL': 0.00,'Sunshade':None},
                                   {'FlagPerimeter':1,'FlagSolarRadiation':2,'direction':12,'FlagUnsteady':1,'name':'LprmdIF', 'AorL': 0.00,'Sunshade':None} ]

    dictionary_element=[]
    for e in StandardHouseElemntsFI:
        temporary=convert_element(e, d)
        if e['name']=='Ceiling':
            temporary['Wall']=d_ceiling
        elif e['name']=='Wall':
            temporary['Wall']=d_wall
        elif e['name']=='Floor':
            temporary['Wall']=d_floor
        elif e['name']=='Window':
            temporary['Window']=d_window
            temporary['Sunshade']=d_sunshade
        elif e['name']=='Door':
            temporary['Door']=d_door
        dictionary_element.append(temporary)

    return {'common':d_common,'element':dictionary_element}

In [14]:
d = {
    'Common' : {
        'Region'                     : 6,
        'MainOccupantRoomFloorArea'  : 30.0,
        'OtherOccupantRoomFloorArea' : 60.0,
        'TotalFloorArea'             : 120.0
    },
    'SimplifiedEnvelopeInput' : {
        'InsulationStructure'        : 1,
        'UCeiling'                   : 0.24,
        'UWall'                      : 0.53,
        'UFloor'                     : 0.48,
        'UDoor'                      : 2.33,
        'UWindow'                    : 3.49,
        'etaWindowCooling'           : 0.51,  
        'etaWindowHeating'           : 0.51,
        'FlagfWinowInput'            : 1,
        'fWindowCooling'             : 0.70,
        'fWindowHeating'             : 0.60,
        'FlagPerimeterInput'         : 1,
        'psiPerimeter'               : 1.80,
        'psiPerimeterDoor'           : 1.80
    }
}

Convert(d)

{'common': {'FlagSimplifiedEnvelopeInput': True,
  'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'element': [{'AorL': 67.8,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'Wall': {'Layers': {'Layers': [{'Cρ': 13,
       'd': 0.179,
       'material': 'GW16K',
       'λ': 0.045},
      {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
    'Ua': 0.24,
    'name': 'Ceiling',
    'structure': 1,
    'type': 1},
   'direction': 1,
   'name': 'Ceiling'},
  {'AorL': 40.63,
   'FlagPerimeter': 1,
   'FlagSolarRadiation': 1,
   'FlagUnsteady': 1,
   'Wall': {'Layers': {'Layers': [{'Cρ': 13,
       'd': 0.076,
       'material': 'GW16K',
       'λ': 0.045},
      {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}]},
    'Ua': 0.53,
    'name': 'Wall',
    'structure': 1,
    'type': 2},
   'direction': 7,
   'name': 'Wall'},
  {'AorL': 29.83,
   'FlagPerimeter': 1,
   'FlagSolarRadiati